Building a GAN model for AI-generated text detection using BERT embeddings.

# Step 0: Prepare the environment and imports

In [1]:
import numpy as np # Для числовых операций
import pandas as pd # Для работы с DataFrame
import random # Для случайных операций (может понадобиться для инициализации)
import string # Для работы со строками (может не использоваться напрямую, но полезно)
import torch # Основная библиотека PyTorch
import torch.nn as nn # Модули для нейронных сетей
import torch.optim as optim # Оптимизаторы для обучения
from torch.utils.data import DataLoader, Dataset # Для загрузки и обработки данных
from transformers import BertTokenizer, BertForSequenceClassification # Токенизатор и предобученная модель BERT
from transformers import BertConfig # Конфигурация для BERT
from transformers.models.bert.modeling_bert import BertEncoder # Энкодер BERT для кастомных архитектур
from sklearn.metrics import roc_auc_score # Метрика AUC для оценки модели
import os # Для работы с файловой системой

# Определение устройства (GPU или CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

print("Этап 0: Подготовка среды и импортов завершен.")

Using device: cuda
Этап 0: Подготовка среды и импортов завершен.


# Step 1: Loading data and setting up paths

In [2]:
import pandas as pd
import os

# Укажем правильный путь к папке sample_data в Colab
COLAB_DATA_DIR = '/content/'

# Загружаем каждый файл напрямую
try:
    src_train_essays = pd.read_csv(os.path.join(COLAB_DATA_DIR, 'train_essays.csv'))
    src_test_essays = pd.read_csv(os.path.join(COLAB_DATA_DIR, 'test_essays.csv'))
    src_train_prompts = pd.read_csv(os.path.join(COLAB_DATA_DIR, 'train_prompts.csv'))
    src_sample_submission = pd.read_csv(os.path.join(COLAB_DATA_DIR, 'sample_submission.csv'))

    # Отобразим информацию о загруженных данных для проверки
    print("src_train_essays head:")
    print(src_train_essays.head())
    print("\nsrc_test_essays head:")
    print(src_test_essays.head())
    print("\nsrc_train_prompts head:")
    print(src_train_prompts.head())
    print("\nsrc_sample_submission head:")
    print(src_sample_submission.head())

    print("\nЭтап 1: Загрузка данных завершена успешно.")

except FileNotFoundError as e:
    print(f"Ошибка: Не удалось найти файл. Убедитесь, что файлы загружены в '{COLAB_DATA_DIR}' и имеют правильные имена.")
    print(f"Детали ошибки: {e}")
except Exception as e:
    print(f"Произошла непредвиденная ошибка: {e}")



src_train_essays head:
         id  prompt_id                                               text  \
0  0059830c          0  Cars. Cars have been around since they became ...   
1  005db917          0  Transportation is a large necessity in most co...   
2  008f63e3          0  "America's love affair with it's vehicles seem...   
3  00940276          0  How often do you ride in a car? Do you drive a...   
4  00c39458          0  Cars are a wonderful thing. They are perhaps o...   

   generated  
0          0  
1          0  
2          0  
3          0  
4          0  

src_test_essays head:
         id  prompt_id          text
0  0000aaaa          2  Aaa bbb ccc.
1  1111bbbb          3  Bbb ccc ddd.
2  2222cccc          4  CCC ddd eee.

src_train_prompts head:
   prompt_id                       prompt_name  \
0          0                   Car-free cities   
1          1  Does the electoral college work?   

                                        instructions  \
0  Write an explanato

# Stage 2: Preparing the BERT model and tokenizer

In [3]:
# Пути для сохранения токенизатора и модели (если нужно сохранять)
tokenizer_save_path = './bert_tokenizer_artifacts'
model_save_path = './bert_model_artifacts'

# Загружаем BERT токенизатор
# 'bert-base-uncased' - это базовая модель BERT без учета регистра
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased') # TODO
print(f"BERT Tokenizer loaded. Vocab size: {tokenizer.vocab_size}")

# Загружаем предобученную модель BertForSequenceClassification
# Мы используем ее только для получения embedding_model и config
pretrained_model = BertForSequenceClassification.from_pretrained('bert-base-uncased').to(device) # TODO
print(f"Pretrained BERT model loaded and moved to {device}.")

# Извлекаем embedding-слой из предобученной модели BERT.
# Это слой, который преобразует входные ID токенов в плотные векторы (эмбеддинги).
embedding_model = pretrained_model.bert.embeddings.to(device) # TODO
print(f"BERT Embedding model extracted and moved to {device}.")
print(f"Embedding dimension (hidden_size): {pretrained_model.config.hidden_size}")

print("\nЭтап 2: Подготовка BERT-модели и токенизатора завершена.")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


BERT Tokenizer loaded. Vocab size: 30522


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Pretrained BERT model loaded and moved to cuda.
BERT Embedding model extracted and moved to cuda.
Embedding dimension (hidden_size): 768

Этап 2: Подготовка BERT-модели и токенизатора завершена.


# Step 3: Defining hyperparameters

In [4]:
# Параметры обучения GAN
train_batch_size = 16 # TODO - Уменьшил для лучшей стабильности и экономии VRAM
test_batch_size = 32  # TODO - Можно сделать больше для ускорения оценки
lr = 0.0002           # TODO - Стандартная скорость обучения для GANs
beta1 = 0.5           # TODO - Параметр бета1 для Adam optimizer (часто 0.5 для GANs)
nz = 100              # Размеры латентного вектора (размерность входного шума для генератора)
num_epochs = 5        # TODO - Количество эпох обучения (может потребоваться больше для лучшего результата)
num_hidden_layers = 6 # TODO - Количество слоев BERT Encoder, используемых в G/D (из 12 для bert-base)
train_ratio = 0.8     # TODO - Соотношение для разделения тренировочных данных на трейн/валидацию

print("Гиперпараметры:")
print(f"  Train Batch Size: {train_batch_size}")
print(f"  Test Batch Size: {test_batch_size}")
print(f"  Learning Rate (LR): {lr}")
print(f"  Beta1 (Adam): {beta1}")
print(f"  Latent Vector Dimension (nz): {nz}")
print(f"  Number of Epochs: {num_epochs}")
print(f"  Number of BERT Hidden Layers (in G/D): {num_hidden_layers}")
print(f"  Train/Validation Split Ratio: {train_ratio}")

print("\nЭтап 3: Определение гиперпараметров завершено.")

Гиперпараметры:
  Train Batch Size: 16
  Test Batch Size: 32
  Learning Rate (LR): 0.0002
  Beta1 (Adam): 0.5
  Latent Vector Dimension (nz): 100
  Number of Epochs: 5
  Number of BERT Hidden Layers (in G/D): 6
  Train/Validation Split Ratio: 0.8

Этап 3: Определение гиперпараметров завершено.


# Step 4: Preparing training data (PyTorch Datasets & DataLoaders)

In [5]:
# ---
## Этап 4: Подготовка данных для обучения (PyTorch Datasets & DataLoaders)
# ---

import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset

# Убедитесь, что src_train_essays определен из предыдущего шага

class GANDAIGDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx], self.labels[idx]

# Определяем общее количество данных из src_train_essays
all_num = len(src_train_essays)

# Вычисляем количество тренировочных и валидационных сэмплов
# train_ratio, train_batch_size, test_batch_size БЕРУТСЯ ИЗ ЭТАПА 3
train_num = int(all_num * train_ratio)
test_num = all_num - train_num

# Разделяем обучающий датасет на тренировочную и валидационную части
train_set = src_train_essays.iloc[:train_num].reset_index(drop=True)
test_set = src_train_essays.iloc[train_num:].reset_index(drop=True)

# Создаем объекты Dataset. .tolist() преобразует Series в список для лучшей совместимости.
# ИСПРАВЛЕНИЕ: Используем 'generated' вместо 'label' для меток
train_dataset = GANDAIGDataset(train_set['text'].tolist(), train_set['generated'].tolist())
test_dataset = GANDAIGDataset(test_set['text'].tolist(), test_set['generated'].tolist())

# Создаем DataLoader'ы
train_loader = DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=test_batch_size, shuffle=False)

print(f"Размер тренировочного набора: {len(train_dataset)} эссе")
print(f"Размер валидационного набора: {len(test_dataset)} эссе")
print(f"Количество батчей в train_loader: {len(train_loader)}")
print(f"Количество батчей в test_loader: {len(test_loader)}")

# Проверка одного батча
for texts, labels in train_loader:
    print(f"\nПример батча из train_loader:")
    print(f"    Тип текстов: {type(texts)}, количество: {len(texts)}")
    print(f"    Первый текст: '{texts[0][:100]}...'")
    print(f"    Тип меток: {type(labels)}, количество: {len(labels)}")
    print(f"    Первая метка: {labels[0]}")
    break

print("\nЭтап 4: Подготовка данных для обучения завершена.")

Размер тренировочного набора: 1102 эссе
Размер валидационного набора: 276 эссе
Количество батчей в train_loader: 69
Количество батчей в test_loader: 9

Пример батча из train_loader:
    Тип текстов: <class 'tuple'>, количество: 16
    Первый текст: 'The using of cars has caused much of the worlds green house gas imitions, in America as much as 50% ...'
    Тип меток: <class 'torch.Tensor'>, количество: 16
    Первая метка: 0

Этап 4: Подготовка данных для обучения завершена.


# Step 5: Defining the Generator Model

In [6]:
# BertConfig для BertEncoder в генераторе и дискриминаторе.
# Используем параметры из предобученной модели, чтобы BertEncoder был совместим.
config = BertConfig(
    vocab_size=tokenizer.vocab_size,
    hidden_size=pretrained_model.config.hidden_size, # 768 для bert-base-uncased
    num_hidden_layers=num_hidden_layers, # Количество слоев, которые мы хотим использовать
    num_attention_heads=pretrained_model.config.num_attention_heads,
    intermediate_size=pretrained_model.config.intermediate_size,
    hidden_act=pretrained_model.config.hidden_act,
    hidden_dropout_prob=pretrained_model.config.hidden_dropout_prob,
    attention_probs_dropout_prob=pretrained_model.config.attention_probs_dropout_prob,
    max_position_embeddings=128, # Явно задаем максимальную длину последовательности
    type_vocab_size=pretrained_model.config.type_vocab_size,
    initializer_range=pretrained_model.config.initializer_range,
    layer_norm_eps=pretrained_model.config.layer_norm_eps,
    position_embedding_type='absolute',
    use_cache=True,
    classifier_dropout=None
)

class Generator(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        # Целевая длина последовательности для эмбеддингов BERT
        self.output_sequence_length = 128
        # Размерность эмбеддингов BERT (768 для bert-base-uncased)
        self.embedding_dim = pretrained_model.config.hidden_size

        # Полносвязный слой, который преобразует nz в больший тензор
        # Его выход будет reshaped в (batch_size, initial_channels, initial_seq_len)
        # Здесь 256 - произвольное количество каналов для начала сверточных слоев.
        # initial_seq_len (например, 8) будет масштабироваться ConvTranspose1d до self.output_sequence_length.
        initial_seq_len = 8 # Например, начинаем с небольшой длины
        self.fc = nn.Linear(input_dim, 256 * initial_seq_len) # TODO

        self.conv_net = nn.Sequential(
            # Input: (batch_size, 256, 8) after fc and reshape
            # Upsample sequence length to 16
            nn.ConvTranspose1d(in_channels=256, out_channels=256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm1d(256),
            nn.ReLU(True),
            # Upsample sequence length to 32
            nn.ConvTranspose1d(in_channels=256, out_channels=128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm1d(128),
            nn.ReLU(True),
            # Upsample sequence length to 64
            nn.ConvTranspose1d(in_channels=128, out_channels=64, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm1d(64),
            nn.ReLU(True),
            # Upsample sequence length to 128
            nn.ConvTranspose1d(in_channels=64, out_channels=self.embedding_dim, kernel_size=4, stride=2, padding=1), # Output 768 channels (embedding_dim)
            nn.BatchNorm1d(self.embedding_dim),
            nn.ReLU(True)
        )
        # BertEncoder будет обрабатывать сгенерированные эмбеддинги
        self.bert_encoder = BertEncoder(config)


    def forward(self, x):
        # Преобразование латентного вектора
        x = self.fc(x)
        # Reshape для ConvTranspose1d: (batch_size, channels, sequence_length)
        # Используем .view() для изменения формы тензора
        x = x.view(x.size(0), 256, 8) # TODO - initial_seq_len (8)
        # Пропуск через сверточную сеть
        x = self.conv_net(x)
        # Permute (транспонирование) для приведения к форме (batch_size, sequence_length, embedding_dim)
        # BertEncoder ожидает такую форму
        x = x.permute(0, 2, 1) # TODO

        # Пропуск через BertEncoder для придания "BERT-подобных" свойств
        # .last_hidden_state содержит выходные эмбеддинги из последнего слоя BertEncoder
        x = self.bert_encoder(x).last_hidden_state # TODO
        return x

# Проверка генератора
test_noise = torch.randn(train_batch_size, nz, device=device)
test_generator = Generator(nz).to(device)
generated_embeddings = test_generator(test_noise)
print(f"Размер сгенерированных эмбеддингов: {generated_embeddings.shape}") # Должно быть (batch_size, 128, 768)

print("\nЭтап 5: Определение модели Генератора завершено.")

Размер сгенерированных эмбеддингов: torch.Size([16, 128, 768])

Этап 5: Определение модели Генератора завершено.


# Step 6: Defining the Discriminator Model

In [7]:
# ---
## Этап 6: Определение модели Дискриминатора
# ---

class SumBertPooler(torch.nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, hidden_states: torch.Tensor) -> torch.Tensor:
        sum_hidden = hidden_states.sum(dim=1)
        sum_mask = sum_hidden.abs().sum(1).unsqueeze(1)
        sum_mask = torch.clamp(sum_mask, min=1e-9)

        mean_embeddings = sum_hidden / sum_mask
        return mean_embeddings

class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.bert_encoder = BertEncoder(config)
        self.bert_encoder.layer = nn.ModuleList([
            layer for layer in pretrained_model.bert.encoder.layer[:num_hidden_layers]
        ])
        self.pooler = SumBertPooler()
        self.classifier = torch.nn.Sequential(
            nn.Linear(pretrained_model.config.hidden_size, 128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(128, 1),
        )

    def forward(self, input_embeddings):
        out = self.bert_encoder(input_embeddings).last_hidden_state
        out = self.pooler(out)
        out = self.classifier(out)
        return torch.sigmoid(out).view(-1)

# Инициализация основной модели дискриминатора
# ВМЕСТО `test_discriminator = Discriminator().to(device)`
# Используем `model = Discriminator().to(device)`
model = Discriminator().to(device)

# Проверка дискриминатора (теперь используем `model`)
# test_noise должен быть определен из Этапа 5 для этой проверки.
# Если `generated_embeddings` уже определен из предыдущего запуска Этапа 5, используй его.
# Если нет, пересоздай его:
# test_noise = torch.randn(train_batch_size, nz, device=device)
# generated_embeddings = test_generator(test_noise) # test_generator должен быть инициализирован
# output_discriminator = model(generated_embeddings)

# Проверка, что модель существует и может обрабатывать входные данные
try:
    # Убедимся, что test_noise и test_generator доступны, или создадим их заново
    if 'test_noise' not in locals() or 'test_generator' not in locals():
        test_noise = torch.randn(train_batch_size, nz, device=device)
        # Assuming Generator class and nz are defined.
        # This part ensures that generated_embeddings is available for testing the Discriminator
        # You would typically define Generator and test_generator in Step 5.
        # If test_generator is not defined yet, this would be `generator = Generator(nz).to(device)`
        # and then `generated_embeddings = generator(test_noise)`
        # For this specific check, we can just make dummy embeddings if needed.
        dummy_embeddings = torch.randn(train_batch_size, 128, pretrained_model.config.hidden_size).to(device)
        output_discriminator = model(dummy_embeddings)
    else:
        output_discriminator = model(generated_embeddings) # Используем сгенерированные ранее эмбеддинги

    print(f"Размер выхода дискриминатора: {output_discriminator.shape}") # Должно быть (batch_size,)
except Exception as e:
    print(f"Ошибка при проверке дискриминатора: {e}")

print("\nЭтап 6: Определение модели Дискриминатора завершено.")

Размер выхода дискриминатора: torch.Size([16])

Этап 6: Определение модели Дискриминатора завершено.


# Step 7: Define support functions for training and assessment

In [8]:
# Функция для оценки AUC
def eval_auc(model):
    model.eval() # Переводим модель в режим оценки (отключаем dropout и Batch Norm для инференса)

    predictions = []
    actuals = []
    with torch.no_grad(): # Отключаем расчет градиентов для оценки, экономит память и ускоряет
        for batch in test_loader:
            texts, label = batch # Разделяем батч на тексты и метки
            # Токенизируем тексты из батча
            encodings = tokenizer(texts, padding='max_length', truncation=True, max_length=128, return_tensors="pt") # TODO
            input_ids = encodings['input_ids'].to(device) # Перемещаем на устройство
            token_type_ids = encodings['token_type_ids'].to(device) # Перемещаем на устройство
            attention_mask = encodings['attention_mask'].to(device) # Перемещаем на устройство


            embeded = embedding_model(input_ids=input_ids, token_type_ids=token_type_ids)


            label = label.float().to(device) # Переводим метки в float и на устройство

            outputs = model(embeded) # Пропускаем эмбеддинги через дискриминатор
            predictions.extend(outputs.cpu().numpy())
            actuals.extend(label.cpu().numpy())

    auc = roc_auc_score(actuals, predictions) # TODO - Вычисляем AUC-ROC
    return auc

# Функция для сохранения информации о модели (для отслеживания лучшей версии)
def get_model_info_dict(model, epoch, auc_score):
    current_device = next(model.parameters()).device
    model.to('cpu') # Переносим модель на CPU перед сохранением state_dict

    model_info = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'auc_score': auc_score,
    }

    model.to(current_device) # Возвращаем модель на исходное устройство
    return model_info

# Функция для подготовки эмбеддингов текста из списка строк
def preparation_embedding(texts):
    # Токенизируем тексты
    encodings = tokenizer(texts, padding='max_length', truncation=True, max_length=128, return_tensors="pt")
    # Перемещаем тензоры на устройство
    input_ids = encodings['input_ids'].to(device) # TODO
    token_type_ids = encodings['token_type_ids'].to(device) # TODO
    # Получаем эмбеддинги. Как и в eval_auc, embedding_model возвращает тензор напрямую.
    embeded = embedding_model(input_ids=input_ids, token_type_ids=token_type_ids) # TODO
    return embeded

print("Этап 7: Определение вспомогательных функций завершено.")


# Проверка функции preparation_embedding
try:
    print("\nТестирование preparation_embedding...")
    sample_texts_for_embedding = [
        "This is a short test sentence for embedding.",
        "Another example text to check the output dimensions."
    ]
    # Используем `sample_texts_for_embedding` для получения эмбеддингов
    test_embeddings = preparation_embedding(sample_texts_for_embedding)
    print(f"Размер эмбеддингов для sample_texts: {test_embeddings.shape}")
    # Ожидаемый размер: (batch_size, max_length, hidden_size)
    # Для bert-base-uncased, hidden_size = 768, max_length = 128
    # Так что ожидаем что-то вроде (2, 128, 768)
    if test_embeddings.shape[0] == len(sample_texts_for_embedding) and test_embeddings.shape[2] == 768:
        print("preparation_embedding: Успех! Размер эмбеддингов соответствует ожиданиям.")
    else:
        print("preparation_embedding: Предупреждение! Размер эмбеддингов не совсем соответствует ожиданиям.")

except Exception as e:
    print(f"Ошибка при тестировании preparation_embedding: {e}")
    print("Убедитесь, что 'tokenizer' и 'embedding_model' корректно инициализированы.")


# Проверка функции eval_auc
try:
    print("\nТестирование eval_auc...")
    # Чтобы eval_auc работал, test_loader должен быть заполнен, а model - это дискриминатор
    # (даже если это заглушка DummyDiscriminator, главное, чтобы он принимал эмбеддинги
    # и выдавал одно число на выходе).
    test_auc_score = eval_auc(model)
    print(f"Предполагаемый AUC score на тестовом наборе: {test_auc_score:.4f}")
    print("eval_auc: Успех! Функция отработала без ошибок.")
    print("Примечание: Значение AUC будет случайным, если модель не обучена.")
except Exception as e:
    print(f"Ошибка при тестировании eval_auc: {e}")
    print("Убедитесь, что 'model' и 'test_loader' корректно инициализированы и их типы данных совместимы.")


print("\n--- Проверки функций Этапа 7 завершены. ---")


# ---
## Проверка функций после Этапа 7
# ---

print("\n--- Проверка функций Этапа 7 ---")

# Проверка функции preparation_embedding
try:
    print("\nТестирование preparation_embedding...")
    sample_texts_for_embedding = [
        "This is a short test sentence for embedding.",
        "Another example text to check the output dimensions."
    ]
    # Используем `sample_texts_for_embedding` для получения эмбеддингов
    test_embeddings = preparation_embedding(sample_texts_for_embedding)
    print(f"Размер эмбеддингов для sample_texts: {test_embeddings.shape}")
    # Ожидаемый размер: (batch_size, max_length, hidden_size)
    # Для bert-base-uncased, hidden_size = 768, max_length = 128
    # Так что ожидаем что-то вроде (2, 128, 768)
    if test_embeddings.shape[0] == len(sample_texts_for_embedding) and test_embeddings.shape[2] == 768:
        print("preparation_embedding: Успех! Размер эмбеддингов соответствует ожиданиям.")
    else:
        print("preparation_embedding: Предупреждение! Размер эмбеддингов не совсем соответствует ожиданиям.")

except Exception as e:
    print(f"Ошибка при тестировании preparation_embedding: {e}")
    print("Убедитесь, что 'tokenizer' и 'embedding_model' корректно инициализированы.")


# Проверка функции eval_auc
try:
    print("\nТестирование eval_auc...")
    # Чтобы eval_auc работал, test_loader должен быть заполнен, а model - это дискриминатор
    # (даже если это заглушка DummyDiscriminator, главное, чтобы он принимал эмбеддинги
    # и выдавал одно число на выходе).
    test_auc_score = eval_auc(model)
    print(f"Предполагаемый AUC score на тестовом наборе: {test_auc_score:.4f}")
    print("eval_auc: Успех! Функция отработала без ошибок.")
    print("Примечание: Значение AUC будет случайным, если модель не обучена.")
except Exception as e:
    print(f"Ошибка при тестировании eval_auc: {e}")
    print("Убедитесь, что 'model' и 'test_loader' корректно инициализированы и их типы данных совместимы.")


print("\n--- Проверки функций Этапа 7 завершены. ---")

Этап 7: Определение вспомогательных функций завершено.

Тестирование preparation_embedding...
Размер эмбеддингов для sample_texts: torch.Size([2, 128, 768])
preparation_embedding: Успех! Размер эмбеддингов соответствует ожиданиям.

Тестирование eval_auc...
Предполагаемый AUC score на тестовом наборе: 0.6073
eval_auc: Успех! Функция отработала без ошибок.
Примечание: Значение AUC будет случайным, если модель не обучена.

--- Проверки функций Этапа 7 завершены. ---

--- Проверка функций Этапа 7 ---

Тестирование preparation_embedding...
Размер эмбеддингов для sample_texts: torch.Size([2, 128, 768])
preparation_embedding: Успех! Размер эмбеддингов соответствует ожиданиям.

Тестирование eval_auc...
Предполагаемый AUC score на тестовом наборе: 0.6073
eval_auc: Успех! Функция отработала без ошибок.
Примечание: Значение AUC будет случайным, если модель не обучена.

--- Проверки функций Этапа 7 завершены. ---


# Step 8: GAN Training Cycle

In [9]:
def GAN_step(optimizerG, optimizerD, netG, netD, real_data_embeddings, label_template, epoch, i):
    # label_template используется для создания новых тензоров меток с правильным размером и устройством

    # (1) Обновление D сети: Максимизация log(D(x)) + log(1 - D(G(z)))
    netD.zero_grad() # Обнуляем градиенты дискриминатора
    batch_size = real_data_embeddings.size(0)

    # Обучение на реальных данных
    label_real = torch.full((batch_size,), 1.0, dtype=torch.float, device=device) # Метки 1 для реальных данных # TODO
    output_real = netD(real_data_embeddings).view(-1)
    errD_real = criterion(output_real, label_real)
    errD_real.backward() # Обратное распространение ошибки для реальных данных
    D_x = output_real.mean().item() # Средний выход дискриминатора для реальных

    # Обучение на фейковых данных, сгенерированных G
    noise = torch.randn(batch_size, nz, device=device) # Генерируем случайный шум
    fake_data_embeddings = netG(noise) # Генерируем фейковые эмбеддинги
    # Метки 0 для фейковых данных
    label_fake = torch.full((batch_size,), 0.0, dtype=torch.float, device=device) # TODO
    # Важно: .detach() отсоединяет фейковые данные от графа вычислений Генератора,
    # чтобы при обновлении Дискриминатора градиенты не текли обратно к Генератору.
    output_fake = netD(fake_data_embeddings.detach()).view(-1)
    errD_fake = criterion(output_fake, label_fake)
    errD_fake.backward() # Обратное распространение ошибки для фейковых данных
    D_G_z1 = output_fake.mean().item() # Средний выход дискриминатора для фейковых (до обновления G)
    errD = errD_real + errD_fake # Общая ошибка дискриминатора
    optimizerD.step() # Обновляем параметры дискриминатора

    # (2) Обновление G сети: Максимизация log(D(G(z)))
    netG.zero_grad() # Обнуляем градиенты генератора
    # Генератор хочет, чтобы дискриминатор считал его выход реальным (т.е., метка 1)
    label_gen = torch.full((batch_size,), 1.0, dtype=torch.float, device=device) # TODO
    output_gen = netD(fake_data_embeddings).view(-1) # Пропускаем фейковые данные через дискриминатор снова
    errG = criterion(output_gen, label_gen) # Ошибка генератора (насколько хорошо он обманул D)
    errG.backward() # Обратное распространение ошибки для генератора
    D_G_z2 = output_gen.mean().item() # Средний выход дискриминатора для фейковых (после обновления G)
    optimizerG.step() # Обновляем параметры генератора

    if i % 50 == 0: # Печатаем прогресс каждые 50 итераций
        print(f'[{epoch+1}/{num_epochs}][{i}/{len(train_loader)}] '
              f'Loss_D: {errD.item():.4f} Loss_G: {errG.item():.4f} '
              f'D(x): {D_x:.4f} D(G(z)): {D_G_z1:.4f} / {D_G_z2:.4f}')

    return optimizerG, optimizerD, netG, netD

# Инициализируем Генератор и Дискриминатор
netG = Generator(nz).to(device) # TODO
netD = Discriminator().to(device) # TODO

# Определяем функцию потерь (Binary Cross-Entropy Loss)
criterion = nn.BCELoss() # TODO

# Определяем оптимизаторы (Adam)
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999)) # TODO
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999)) # TODO

model_infos = [] # Список для хранения информации о моделях (для выбора лучшей по AUC)
best_auc = 0.0 # Отслеживание лучшего AUC
max_auc_model_info = None # Хранение state_dict лучшей модели

print("Начало обучения GAN...")
for epoch in range(num_epochs):
    netG.train() # Переводим генератор в режим обучения
    netD.train() # Переводим дискриминатор в режим обучения
    for i, data in enumerate(train_loader, 0):
        # data[0] - это тексты (str), data[1] - это метки (int)

        with torch.no_grad(): # Градиенты для эмбеддингов не нужны
            # Подготовка эмбеддингов реальных данных
            # data[0] содержит тексты, которые нужно токенизировать и эмбеддировать
            real_data_embeddings = preparation_embedding(data[0])

        optimizerG, optimizerD, netG, netD = GAN_step(
            optimizerG=optimizerG, # TODO
            optimizerD=optimizerD, # TODO
            netG=netG,
            netD=netD,
            real_data_embeddings=real_data_embeddings, # TODO
            label_template=data[1].float().to(device), # TODO - Используем метки из даталоадера как шаблон
            epoch=epoch, i=i)

    # Оценка модели (Дискриминатора) после каждой эпохи
    auc_score = eval_auc(netD) # TODO
    print(f"\n--- Epoch {epoch+1}/{num_epochs} --- Validation AUC: {auc_score:.4f} ---")

    current_model_info = get_model_info_dict(netD, epoch, auc_score)
    if auc_score > best_auc: # Если текущий AUC лучше предыдущего лучшего
        best_auc = auc_score
        max_auc_model_info = current_model_info # Обновляем информацию о лучшей модели

    model_infos.append(current_model_info)

print('Обучение GAN завершено！')

Начало обучения GAN...
[1/5][0/69] Loss_D: 1.3879 Loss_G: 0.7357 D(x): 0.4794 D(G(z)): 0.4794 / 0.4792
[1/5][50/69] Loss_D: 1.3086 Loss_G: 0.7550 D(x): 0.5101 D(G(z)): 0.4703 / 0.4700

--- Epoch 1/5 --- Validation AUC: 0.3164 ---
[2/5][0/69] Loss_D: 1.2630 Loss_G: 0.7722 D(x): 0.5261 D(G(z)): 0.4625 / 0.4620
[2/5][50/69] Loss_D: 1.1003 Loss_G: 0.8637 D(x): 0.5763 D(G(z)): 0.4226 / 0.4216

--- Epoch 2/5 --- Validation AUC: 0.9200 ---
[3/5][0/69] Loss_D: 1.0235 Loss_G: 0.9191 D(x): 0.5992 D(G(z)): 0.4003 / 0.3989
[3/5][50/69] Loss_D: 0.8091 Loss_G: 1.1155 D(x): 0.6642 D(G(z)): 0.3296 / 0.3278

--- Epoch 3/5 --- Validation AUC: 0.0364 ---
[4/5][0/69] Loss_D: 0.7281 Loss_G: 1.2069 D(x): 0.6905 D(G(z)): 0.3008 / 0.2991
[4/5][50/69] Loss_D: 0.5423 Loss_G: 1.4663 D(x): 0.7568 D(G(z)): 0.2317 / 0.2308

--- Epoch 4/5 --- Validation AUC: 0.0291 ---
[5/5][0/69] Loss_D: 0.4827 Loss_G: 1.5708 D(x): 0.7800 D(G(z)): 0.2088 / 0.2079
[5/5][50/69] Loss_D: 0.3545 Loss_G: 1.8540 D(x): 0.8322 D(G(z)): 0.15

Step 9: Inference and Submission File Creation

In [10]:
# ---
## Этап 9: Инференс и создание файла submission
# ---

print("\nНачало этапа инференса...")

# Проверка наличия max_auc_model_info и model_infos
if max_auc_model_info is None:
    print("Внимание: max_auc_model_info не был установлен. Возможно, обучение не завершилось или AUC не рос.")
    print("Использую информацию о последней обученной модели для инференса.")
    if model_infos:
        # Берем последнюю сохраненную модель, если лучшая не найдена
        max_auc_model_info = model_infos[-1]
    else:
        # Если model_infos пуст, значит, обучение не производилось вовсе
        raise ValueError("Нет доступной информации о моделях для инференса. Обучение, возможно, не производилось.")

# Загружаем лучшую модель дискриминатора
# model уже был инициализирован как Discriminator().to(device) в Этапе 6.
# Можно перезагрузить его state_dict, или создать новый экземпляр, как ты и делаешь здесь.
# Создание нового экземпляра:
inference_model = Discriminator().to(device) # Создаем новый экземпляр дискриминатора для инференса
inference_model.load_state_dict(max_auc_model_info['model_state_dict']) # Загружаем сохраненные веса
inference_model.eval() # Переводим модель в режим оценки (отключаем дропаут и BatchNorm для инференса)

print(f"Лучшая модель Дискриминатора (AUC: {max_auc_model_info.get('auc_score', 'N/A'):.4f}) загружена.")

# Подготовка данных для инференса из src_test_essays
# Убедитесь, что src_test_essays загружен на Этапе 1
inference_texts = src_test_essays['text'].tolist()
inference_ids = src_test_essays['id'].tolist()

class InferenceDataset(torch.utils.data.Dataset):
    def __init__(self, texts):
        self.texts = texts

    def __getitem__(self, idx):
        return self.texts[idx]

    def __len__(self):
        return len(self.texts)

sub_dataset = InferenceDataset(inference_texts)

# Используем test_batch_size, определенный в Этапе 3
inference_loader = DataLoader(sub_dataset, batch_size=test_batch_size, shuffle=False)

sub_predictions = []
with torch.no_grad(): # Отключаем градиенты
    for batch_texts in inference_loader:
        # Токенизируем тексты
        encodings = tokenizer(batch_texts, padding='max_length', truncation=True, max_length=128, return_tensors="pt")
        input_ids = encodings['input_ids'].to(device)
        token_type_ids = encodings['token_type_ids'].to(device)
        # Получаем эмбеддинги. ВНИМАНИЕ: УБРАНО .last_hidden_state
        embeded = embedding_model(input_ids=input_ids, token_type_ids=token_type_ids)

        outputs = inference_model(embeded) # Пропускаем через загруженную модель дискриминатора
        sub_predictions.extend(outputs.cpu().numpy()) # Сохраняем предсказания

# Создаем DataFrame для submission файла
# sub_predictions уже должен быть одномерным массивом благодаря .view(-1) в Discriminator
sub_ans_df = pd.DataFrame({'id': inference_ids, 'score': sub_predictions})

print("\nПример финального submission DataFrame:")
print(sub_ans_df.head())

# Сохранение submission файла
submission_filename = 'submission.csv'
sub_ans_df.to_csv(submission_filename, index=False)
print(f"\nФайл {submission_filename} успешно создан.")

print("\nЭтап 9: Инференс и создание файла submission завершен.")


Начало этапа инференса...
Лучшая модель Дискриминатора (AUC: 0.9200) загружена.

Пример финального submission DataFrame:
         id     score
0  0000aaaa  0.598717
1  1111bbbb  0.598734
2  2222cccc  0.598745

Файл submission.csv успешно создан.

Этап 9: Инференс и создание файла submission завершен.


# SUM

Step 0: Prepare environment and imports. Set up the Colab environment and downloaded all necessary libraries.

Step 1: Load data. Ensured that the competition data loads correctly.

Step 2: Prepare BERT model and tokenizer. Initialized the tokenizer and extracted the BERT embedding layer for use in the GAN.

Step 3: Define hyperparameters. Centrally defined all key parameters for training the model.

Step 4: Prepare training data. Created a custom Dataset and DataLoader to efficiently feed data to the model. Fixed the issue with duplicate hyperparameters.

Step 5: Define the Generator model. Designed the Generator architecture that transforms random noise into BERT-like embeddings.

Step 6: Define the Discriminator model. Created a Discriminator that uses a BERT encoder to classify embeddings as "real" or "fake". Ensured that the model is globally accessible.

Step 7: Helper functions. Implemented functions for AUC estimation and embedding preparation, eliminating the nuances of last_hidden_state.

Step 8: GAN training loop. Set up the logic for step-by-step training of the Generator and Discriminator, including loss functions and optimizers.

Step 9: Inference and submission file generation. Successfully made predictions on the test dataset and generated a submission.csv file with probabilities for each essay.